In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    
from torchvision import models, transforms
import logging
import requests
from io import BytesIO
import os
import cv2

import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.utils
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import shutil
import os

In [ ]:
#for Plotting training graphs
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

Dataloader

In [ ]:
#dataloader for getting triplets from dataset
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        

    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)

        #search until requirements are met (same class)
        while True:
          img1_tuple = random.choice(self.imageFolderDataset.imgs)
          if img0_tuple[1] == img1_tuple[1]:
            break
        while True:
          #search until they arent the same class
          img2_tuple = random.choice(self.imageFolderDataset.imgs)
          if img0_tuple[1] != img2_tuple[1]:
            break
        
        
        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img2 = Image.open(img2_tuple[0])

      
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)


        return img0, img1, img2
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Transformations

In [ ]:
# Resize the images and transform to tensors
transformation = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=[0.486, 0.459, 0.408],
                                 std=[0.229, 0.224, 0.225]),
                                    
                                    ])

In [ ]:
folder_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Person_reID_baseline_pytorch/Market-1501-v15.09.15/pytorch/train_all")
folder_validationset = datasets.ImageFolder(root="//content/drive/MyDrive/Person_reID_baseline_pytorch/Market-1501-v15.09.15/pytorch/valid")


In [ ]:
train = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transformation)

valid = SiameseNetworkDataset(imageFolderDataset=folder_validationset,
                                        transform=transformation)


In [ ]:
trainloader = DataLoader(train, batch_size=32, num_workers=0, shuffle=True)
validloader = DataLoader(valid, batch_size=32, num_workers=0, shuffle=False)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
from torchvision.models.efficientnet import efficientnet_b1
class Tripletb1(torch.nn.Module):
    def __init__(self):
        super(Tripletb1, self).__init__()
        self.net = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights)

    def forward(self, input1, input2, input3):

        output1 = self.net(input1)
        output2 = self.net(input2)
        output3 = self.net(input3)

        return output1, output2, output3

In [ ]:
# define the tripletLoss function
class TripletLoss(torch.nn.Module):
    """
    Quadruplet loss function.
    Builds on the Triplet Loss and takes 4 data input: one anchor, one positive and two negative examples. The negative examples needs not to be matching the anchor, the positive and each other.
    the margins adds a minimal distance the model has to find between the images, instead of just >0
    """
    def __init__(self, margin1=1):
        super(TripletLoss, self).__init__()
        self.margin1 = margin1

    def forward(self, anchor, positive, negative1):

        squarred_distance_pos = (anchor - positive).pow(2).sum(1)
        squarred_distance_neg = (anchor - negative1).pow(2).sum(1)

        tripletloss = \
            F.relu(self.margin1 + squarred_distance_pos - squarred_distance_neg) 

        return tripletloss.mean()

start training only with the loaded wieghts


In [ ]:
net = Tripletb1().cuda()
criterion = TripletLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0004)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',0.5)

Load the checkpoints to resume training if halted


In [ ]:
loss_historytrain = [] 
loss_historyvalid = [] 
loss_train_items =[]

min_valid_loss = 100
epochs = []

# Iterate throught the epochs
for epoch in range(60):
    torch.cuda.empty_cache()
    net.train()
    train_loss = 0.0
    epochs.append(epoch)
    # Iterate over batches
    for i, (img0, img1, img2) in enumerate(trainloader,0):

        # Send the images to CUDA
        img0, img1, img2 = img0.cuda(), img1.cuda(), img2.cuda()

        # Zero the gradients
        optimizer.zero_grad(set_to_none=True)

      
        # Pass in the three images into the network 
        output1, output2, output3= net(img0, img1, img2)

        # Pass the outputs into the loss function
        loss_siamese = criterion(output1, output2, output3)

        # Calculate the backpropagation
        loss_siamese.backward()

        # Optimize
        optimizer.step()

        # add the loss to the total
        
        train_loss += loss_siamese.item()
        
        

        
        loss_train_items.append(loss_siamese.item())
        if i % 10 == 0 and i>0:
          average = sum(loss_train_items[-9:]) / 10
          print('average last 10 iterations = ', sum(loss_train_items[-9:]) / 10)

    
    epoch_loss = (train_loss /len(trainloader))
    loss_historytrain.append(float(epoch_loss))
    print('average loss epoch =', (train_loss /len(trainloader)))
    torch.cuda.empty_cache()


    valid_loss= 0.0
    # Iterate over batches
    with torch.no_grad():
      net.eval()
      for i, (img0, img1, img2) in enumerate(validloader,0):

          # Send the images to CUDA
          img0, img1, img2 = img0.cuda(), img1.cuda(), img2.cuda()

        
          # Pass in the three images into the network 
          output1, output2, output3= net(img0, img1, img2)

          # Pass the outputs into the loss function
          loss_valid = criterion(output1, output2, output3)

          # Every 10 batch print out the loss
          
          valid_loss += loss_valid.item()

          
                  

   
            
  
      print('average valid loss epoch =', (valid_loss /len(validloader)))
      average_valid_loss = valid_loss /len(validloader)
      loss_historyvalid.append(float(average_valid_loss))
      scheduler.step(average_valid_loss)
      if min_valid_loss > average_valid_loss:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{average_valid_loss:.6f}) \t Saving The Model')
          min_valid_loss = average_valid_loss
          # Saving State Dict
          torch.save({
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 
           'triplet1501V2validb1schedule')

     
      
      torch.cuda.empty_cache()

show_plot(epochs,loss_historytrain)
show_plot(epochs,loss_historyvalid)